# Doing an exploration on recent disaster tweets for a disaster category

In [1]:
import pandas as pd
import pickle

import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import networkx as nx 
import nx_altair as nxa
from collections import Counter
import warnings

from tools import extract_assign_location, disaster_title, topNlocations, top_locations, \
                    sampleNtweets, top_locations_tweets, show_top_locations, get_top_locations_flatten, \
                    get_location_graph_data, build_network, get_layout, plot_disaster_mentions, is_intersect, \
                    show_tweets_locations, top_location_pairs

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 280)

## Define any upstream dependencies

In [2]:
upstream = [
    'vectorizer',
    'category_classification_models',
    'recent_tweets_wildfire',
    'recent_tweets_earthquake',
    'recent_tweets_flood',
    'recent_tweets_hurricane'
]
random_seed = None
disaster_kind = None

In [3]:
# Parameters
random_seed = 42
disaster_kind = "recent_tweets_wildfire"
upstream = {
    "vectorizer": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.ipynb",
        "vectorizer": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer.pkl",
        "vocab": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vocab.pkl",
    },
    "category_classification_models": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/category_classification_models.ipynb",
        "model_lr": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_lr.pkl",
        "model_rf": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_rf.pkl",
        "model_nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_nb.csv",
        "model_votingc": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/model_votingc.csv",
    },
    "recent_tweets_earthquake": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_earthquake.csv",
    },
    "recent_tweets_wildfire": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_wildfire.csv",
    },
    "recent_tweets_hurricane": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_hurricane.csv",
    },
    "recent_tweets_flood": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.ipynb",
        "file": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/recent_tweets_flood.csv",
    },
}
product = {
    "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/tweets_timeline_fire.ipynb"
}


In [4]:
print('Disaster kind: ', disaster_title(disaster_kind))

Disaster kind:  Wildfire


## Load the sample of recent tweets

In [5]:
df_tweets = pd.read_csv(upstream[disaster_kind]['file'])
df_tweets.sample(5)

,author_id,tweet_text,created_at,hashtags,users
79796,18079519,"“What led him to stop and notice a spider and decide to try to burn it, we don’t know."" https://t.co/nPixSoXXyL",2022-08-03 01:40:00+00:00,[],[]
10822,15809727,RT @JoeRich44: Fire Dejoy NOW! \n\nhttps://t.co/qmJEemlHh7,2022-08-05 20:09:47+00:00,[],['JoeRich44']
111269,1399835083326980099,‘Devastating’: McKinney Fire destroys Klamath River community hub as California wildfire rages #Redding #News https://t.co/59pcwIH8MA,2022-08-01 23:02:29+00:00,"['Redding', 'News']",[]
22079,1301482543032815619,RT @AJEnglish: A small town in the state of Washington was evacuated due to a fast-moving fire that burned a half-dozen homes https://t.co/…,2022-08-06 00:34:08+00:00,[],['AJEnglish']
142227,35151043,"RT @HotshotWake: A Monster: Klamath, The McKinney Fire. #california #mckinneyfire #wildfire #fire #cali #ca #calfire https://t.co/QWwWsX4shN",2022-08-01 00:40:16+00:00,"['california', 'mckinneyfire', 'wildfire', 'fire', 'cali', 'ca', 'calfire']",['HotshotWake']


## Extract the locations from the tweet sample

In [6]:
%%time
df_tweets, df_locs_most_common = extract_assign_location(df_tweets, number_most_common=10)

df_tweets.sample(5)

0it [00:00, ?it/s]

22it [00:00, 215.92it/s]

56it [00:00, 284.00it/s]

88it [00:00, 299.79it/s]

124it [00:00, 322.94it/s]

...

143821it [06:35, 366.08it/s]

143858it [06:35, 365.55it/s]

143881it [06:35, 363.39it/s]

CPU times: user 7min 18s, sys: 4.99 s, total: 7min 23s
Wall time: 7min 22s


,author_id,tweet_text,created_at,hashtags,users,location
61371,278405280,RT @KREM2: Rapidly growing fire near Williams Lake; level 3 evacuations in place https://t.co/ZAdGwSBmzw,2022-08-03 23:00:50+00:00,[],['KREM2'],[Williams Lake]
69579,943910347,"RT @Firewise: Know your #HomeIgnitionZone: In the Extended Zone (30-100ft from the home), the goal is to interrupt the path of #wildfire by…",2022-08-03 18:37:54+00:00,"['HomeIgnitionZone', 'wildfire']",['Firewise'],[Extended Zone]
1871,3300720066,"RT @ABC: A wildfire that was intentionally set, according to authorities, caused damage to vehicles in northwest Spain.\n\nOfficials said the…",2022-08-05 15:26:12+00:00,[],['ABC'],[Spain]
121603,209088136,"Two bodies were found inside a burned vehicle as crews battled the McKinney Fire, California's largest wildfire this year. https://t.co/nxTVDiey6D",2022-08-01 16:33:49+00:00,[],[],[California]
48162,314024592,RT @OSUFireProgram: Our Fire Specialist in NE OR recommends that regardless of a risk rating from the many many models out there (!!!) peop…,2022-08-04 22:43:53+00:00,[],['OSUFireProgram'],[]


In [7]:
df_locs_most_common

,location,count
0,California,19750
1,Utah,10058
2,Northern California,7988
3,Klamath National Forest,4293
4,Oregon,3476
5,Northern Cali,3165
6,Washington,2591
7,California’s,2161
8,McKinney Fire,1906
9,Texas,1437


## Predict the class labels

In [8]:
# Get the vectorizer
with open(upstream["vectorizer"]["vectorizer"], 'rb') as f:
    vectorizer = pickle.load(f)

In [9]:
# Get the classification model
with open(upstream["category_classification_models"]["model_lr"], 'rb') as f:
    classifier = pickle.load(f)

In [10]:
%%time
# Predict the class labels
class_label = classifier.predict(vectorizer.transform(df_tweets['tweet_text']))
df_tweets['class_label'] = class_label

df_tweets.sample(5)

CPU times: user 34 s, sys: 111 ms, total: 34.1 s
Wall time: 34.1 s


,author_id,tweet_text,created_at,hashtags,users,location,class_label
67204,741760321,"RT @washingtonpost: “Don’t do drugs kids (and don’t start spiders on fire during a drought),” Utah Gov. Spencer Cox tweeted Tuesday in resp…",2022-08-03 21:52:33+00:00,[],['washingtonpost'],[Tuesday],other_relevant_information
94340,3157726788,A man has been arrested after he allegedly started a wildfire in Utah while trying to burn a spider. https://t.co/1UyqOFrsy2,2022-08-03 00:17:22+00:00,[],[],[Utah],other_relevant_information
46766,1519362092066824194,Seems reasonable https://t.co/00qCNwUhOP,2022-08-04 19:23:25+00:00,[],[],[],not_humanitarian
33878,329942101,"On high fire danger days, avoid any activities that could spark a wildfire including the leading causes of fire escapes in Montana: debris burning, escaped campfires, and lawn/farm equipment. Learn how to prevent wildfires at https://t.co/X2qOqTSeFK #fireyear2022 #mtfire",2022-08-04 16:31:00+00:00,"['fireyear2022', 'mtfire']",[],[Montana],other_relevant_information
127177,17172801,RT @NOAASatellitePA: NEW: @NOAA's #GOES17🛰️ was watching as California's #McKinneyFire showed explosive growth on Saturday. The #wildfire h…,2022-08-01 13:26:26+00:00,"['GOES17', 'McKinneyFire', 'wildfire']","['NOAASatellitePA', 'NOAA']",[],other_relevant_information


## Plot the intensity for each catagory, and for a particular location

### Prepare the time series grouped by disaster class label 

In [11]:
df_tweets['created_at'] = pd.to_datetime(df_tweets['created_at'], errors='coerce')
df_plot = df_tweets.copy()

df_plot['created_at'] = df_plot['created_at'].dt.floor('8H') # Round to 8 hours
df_plot['count'] = 1
df_group_plot = df_plot[['created_at', 'class_label', 'count']].groupby(['created_at', 'class_label']).count().reset_index()
df_group_plot['created_at'] = pd.to_datetime(df_group_plot['created_at'])
df_group_plot.head()

,created_at,class_label,count
0,2022-07-31 08:00:00+00:00,caution_and_advice,74
1,2022-07-31 08:00:00+00:00,displaced_people_and_evacuations,70
2,2022-07-31 08:00:00+00:00,infrastructure_and_utility_damage,41
3,2022-07-31 08:00:00+00:00,not_humanitarian,43
4,2022-07-31 08:00:00+00:00,other_relevant_information,948


### Prepare the time series grouped by location label 

In [12]:
df_explode = df_tweets.copy()

most_common10 = df_locs_most_common.head(10)['location'].tolist()

df_explode = df_explode.explode('location').reset_index(drop=True)
df_explode.dropna(inplace=True)
df_explode['created_at'] = df_explode['created_at'].dt.floor('8H') # Round to 8 hours
df_explode['count'] = 1
df_group_explode = df_explode[['created_at', 'location', 'count']].groupby(
    ['created_at', 'location']
).count().reset_index()
df_group_explode = df_group_explode[df_group_explode['location'].isin(most_common10)]
df_group_explode.head()

,created_at,location,count
7,2022-07-31 08:00:00+00:00,California,294
9,2022-07-31 08:00:00+00:00,California’s,2
29,2022-07-31 08:00:00+00:00,Klamath National Forest,6
36,2022-07-31 08:00:00+00:00,McKinney Fire,3
52,2022-07-31 08:00:00+00:00,Northern California,12


### Find the top 10 locations in the tweets sample

In [13]:
top10_locs = list(df_group_explode['location'].unique())
top10_locs

['California',
 'California’s',
 'Klamath National Forest',
 'McKinney Fire',
 'Northern California',
 'Oregon',
 'Texas',
 'Washington',
 'Utah',
 'Northern Cali']

In [14]:
chart = alt.Chart(df_group_explode[df_group_explode['location'].isin(top10_locs)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='location:N',
    tooltip=['location', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
).interactive()

alt.Chart(...)

## Combine the most important locations per category

In [15]:
df_top_locations = top_locations(df_tweets, top10_locs)
df_top_locations.sample(5)

,created_at,class_label,locations,count
386,2022-08-06 04:00:00+00:00,other_relevant_information,"[California, Utah, Washington]",30
341,2022-08-05 10:00:00+00:00,displaced_people_and_evacuations,[Northern California],17
369,2022-08-05 20:00:00+00:00,rescue_volunteering_or_donation_effort,[Texas],28
253,2022-08-04 00:00:00+00:00,caution_and_advice,[California],20
263,2022-08-04 02:00:00+00:00,not_humanitarian,[Utah],39


### Define the class labels we are interested in

In [16]:
class_labels = [
    'displaced_people_and_evacuations',
    'injured_or_dead_people',
    'rescue_volunteering_or_donation_effort',
    'requests_or_urgent_needs',
    'sympathy_and_support',
    'infrastructure_and_utility_damage',
    'caution_and_advice'
]

In [17]:
chart = alt.Chart(df_top_locations[df_top_locations['class_label'].isin(class_labels)]).mark_line(size=2).encode(
    x='created_at:T',
    y='count:Q',
    color='class_label:N',
    tooltip=['class_label:N', 'locations', 'count']
)

chart.properties(
    width=500,
    height=500
).properties(
    title=disaster_title(disaster_kind) + ' disasters in the world, interactive chart'
)

alt.Chart(...)

## Show sample tweets per most important locations above

In [18]:
df_top_location_tweets = top_locations_tweets(df_tweets, top10_locs)
df_top_location_tweets.sample(5)

,created_at,class_label,tweet_text,locations,count
147,2022-08-02 12:00:00+00:00,displaced_people_and_evacuations,"[RT @gulf_news: #Photos: #California's largest fire this year forces thousands to evacuate\nMcKinney Fire is now California's largest wildfir…, RT @RadioNLNews: UPDATE: @cityofkamloops says its crews are monitoring ""Watching Creek"" wildfire after regional district issues evac...","[California, McKinney Fire, Oregon]",40
51,2022-08-01 04:00:00+00:00,not_humanitarian,"[RT @HotshotWake: California: The McKinney fire was mapped at just under 30,000 acres. By the looks of it that’ll grow. The other two fires…, RT @USATODAY: The McKinney Fire erupted Friday in the Klamath National Forest and quickly burned out of control, growing to over 50,00...",[Klamath National Forest],7
189,2022-08-03 02:00:00+00:00,caution_and_advice,"[RT @OSFM: We've got some really dry conditions across Oregon. Please do what you can to avoid sparking a fire. \n\nOne tip, avoid driving or…, @CA_DWR, @saveourwater, &amp; @CAL_FIRE teamed up to provide Californians with recs. on how to be fire safe while conserving water t...",[Oregon],9
53,2022-08-01 06:00:00+00:00,caution_and_advice,"[RT @CastanetKam: As fire risk rises, BC Wildfire Service faces retention issues https://t.co/LB6Uh3aG4s https://t.co/DNPnExweSl, Thunderstorms w lightning but little to no rain are expected in Northern #California over the next few days. ❗️Let’s hope it’s not a repeat of 202...",[California],6
134,2022-08-02 06:00:00+00:00,other_relevant_information,"[RT @axios: The McKinney Fire, California's largest wildfire this year, remained 0% contained as of this morning. So far, it has burned near…, RT @BCGovFireInfo: The Polygon Pond wildfire (K20956), 7.5km west of Kamloops, is now classified as Being Held at three hectares in s...","[California, California’s, Northern California]",264


In [19]:
show_top_locations(df_top_location_tweets, class_labels, disaster_kind)

alt.VConcatChart(...)

## Visualize the locations and relationship between them as a network

### Select the top 20 locations for this disaster

In [20]:
loc20 = get_top_locations_flatten(df_tweets, 20)
loc20

['California',
 'Utah',
 'Northern California',
 'Klamath National Forest',
 'Oregon',
 'Northern Cali',
 'Washington',
 'California’s',
 'McKinney Fire',
 'Texas',
 'Keremeos Creek',
 'US',
 'New Mexico',
 'North Texas',
 'Kamloops',
 'Kentucky',
 'Lind',
 'Siskiyou County',
 'U.S.',
 'grants f…']

### Get the tweets for just the top 20 locations

In [21]:
df_loc_graph = get_location_graph_data(df_tweets, loc20)
df_loc_graph.sample(10)

,class_label,location_x,location_y,count
7,displaced_people_and_evacuations,Oregon,McKinney Fire,378
88,other_relevant_information,Washington,Lind,200
57,other_relevant_information,Klamath National Forest,California’s,7
104,sympathy_and_support,California’s,California,10
51,not_humanitarian,Klamath National Forest,California,9
21,infrastructure_and_utility_damage,Siskiyou County,California’s,5
45,injured_or_dead_people,Siskiyou County,Northern California,18
82,other_relevant_information,U.S.,California,12
66,other_relevant_information,Northern California,Klamath National Forest,277
56,other_relevant_information,Klamath National Forest,California,210


In [22]:
plot_disaster_mentions(df_loc_graph, 'caution_and_advice', 'Caution And Advice')

alt.LayerChart(...)

### Get the most frequent location pair

In [23]:
loc_pairs = top_location_pairs(df_loc_graph, 'caution_and_advice', top_pairs_count=1)
loc_pairs

[['Klamath National Forest', 'California']]

### Show a sample for this location pair

In [24]:
show_tweets_locations(df_tweets, class_label='caution_and_advice' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,RT @NBCNightlyNews: California is in a state of emergency as the McKinney Fire rips through the Klamath National Forest and expanded rapidl…,"[California, Klamath National Forest]"
1,RT @NBCNightlyNews: California is in a state of emergency as the McKinney Fire rips through the Klamath National Forest and expanded rapidl…,"[California, Klamath National Forest]"
2,RT @NBCNightlyNews: California is in a state of emergency as the McKinney Fire rips through the Klamath National Forest and expanded rapidl…,"[California, Klamath National Forest]"
3,RT @NBCNightlyNews: California is in a state of emergency as the McKinney Fire rips through the Klamath National Forest and expanded rapidl…,"[California, Klamath National Forest]"
4,RT @NBCNightlyNews: California is in a state of emergency as the McKinney Fire rips through the Klamath National Forest and expanded rapidl…,"[California, Klamath National Forest]"


In [25]:
plot_disaster_mentions(df_loc_graph, 'displaced_people_and_evacuations', 'Displaced People And Evacuations')

alt.LayerChart(...)

### Get the most frequent location pair

In [26]:
loc_pairs = top_location_pairs(df_loc_graph, 'displaced_people_and_evacuations', top_pairs_count=1)
loc_pairs

[['Oregon', 'California']]

### Show a sample for this location pair

In [27]:
show_tweets_locations(df_tweets, class_label='displaced_people_and_evacuations' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"RT @Reuters: The fast-moving McKinney Fire in northern California near the Oregon border has forced 2,000 residents to evacuate to a Red Cr…","[McKinney Fire, California, Oregon]"
1,"RT @Reuters: The fast-moving McKinney Fire in northern California near the Oregon border has forced 2,000 residents to evacuate to a Red Cr…","[McKinney Fire, California, Oregon]"
2,"RT @Reuters: The fast-moving McKinney Fire in northern California near the Oregon border forced 2,000 residents to evacuate and destroyed h…","[McKinney Fire, California, Oregon]"
3,"RT @Reuters: The fast-moving McKinney Fire in northern California near the Oregon border forced 2,000 residents to evacuate and destroyed h…","[McKinney Fire, California, Oregon]"
4,"RT @Reuters: The fast-moving McKinney Fire in northern California near the Oregon border forced 2,000 residents to evacuate and destroyed h…","[McKinney Fire, California, Oregon]"


In [28]:
plot_disaster_mentions(df_loc_graph, 'injured_or_dead_people', 'Injured Or Dead People')

alt.LayerChart(...)

### Get the most frequent location pair

In [29]:
loc_pairs = top_location_pairs(df_loc_graph, 'injured_or_dead_people', top_pairs_count=1)
loc_pairs

[['Siskiyou County', 'California’s']]

### Show a sample for this location pair

In [30]:
show_tweets_locations(df_tweets, class_label='injured_or_dead_people' , locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"RT @NBCNews: Two people were found dead in a burned-out vehicle by firefighters battling California’s largest wildfire, the Siskiyou County…","[California’s, Siskiyou County]"
1,"RT @NBCNews: Two people were found dead in a burned-out vehicle by firefighters battling California’s largest wildfire, the Siskiyou County…","[California’s, Siskiyou County]"
2,"Two people were found dead in a burned-out vehicle by firefighters battling California’s largest wildfire, the Siskiyou County Sheriff’s Office says. https://t.co/nD1EhgLBed","[California’s, Siskiyou County]"
3,"RT @NBCNews: Two people were found dead in a burned-out vehicle by firefighters battling California’s largest wildfire, the Siskiyou County…","[California’s, Siskiyou County]"
4,"RT @NBCNews: Two people were found dead in a burned-out vehicle by firefighters battling California’s largest wildfire, the Siskiyou County…","[California’s, Siskiyou County]"


In [31]:
plot_disaster_mentions(df_loc_graph, 'rescue_volunteering_or_donation_effort',
                       'Rescue Volunteering Or Donation Effort')

alt.LayerChart(...)

### Get the most frequent location pair

In [32]:
loc_pairs = top_location_pairs(df_loc_graph, 'rescue_volunteering_or_donation_effort', top_pairs_count=1)
loc_pairs

[['Siskiyou County', 'California']]

### Show a sample for this location pair

In [33]:
show_tweets_locations(df_tweets, class_label='rescue_volunteering_or_donation_effort',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"RT @Earth42morrow: California Fire: McKinney Fire in Siskiyou County explodes to over 18,000 acres \n\n#California #Wildfire #McKinney #Siski…","[Siskiyou County, California]"
1,"RT @Earth42morrow: California Fire: McKinney Fire in Siskiyou County explodes to over 18,000 acres \n\n#California #Wildfire #McKinney #Siski…","[Siskiyou County, California]"
2,"RT @Earth42morrow: California Fire: McKinney Fire in Siskiyou County explodes to over 18,000 acres \n\n#California #Wildfire #McKinney #Siski…","[Siskiyou County, California]"
3,"RT @Earth42morrow: California Fire: McKinney Fire in Siskiyou County explodes to over 18,000 acres \n\n#California #Wildfire #McKinney #Siski…","[Siskiyou County, California]"
4,"RT @Earth42morrow: California Fire: McKinney Fire in Siskiyou County explodes to over 18,000 acres \n\n#California #Wildfire #McKinney #Siski…","[Siskiyou County, California]"


In [34]:
plot_disaster_mentions(df_loc_graph, 'requests_or_urgent_needs',
                       'requests_or_urgent_needs')

'No records found for class requests_or_urgent_needs'

### Get the most frequent location pair

In [35]:
loc_pairs = top_location_pairs(df_loc_graph, 'requests_or_urgent_needs', top_pairs_count=1)
loc_pairs

[[]]

### Show a sample for this location pair

In [36]:
show_tweets_locations(df_tweets, class_label='requests_or_urgent_needs',
                      locations=loc_pairs[0], sample=5)

'No location pairs provided'

In [37]:
plot_disaster_mentions(df_loc_graph, 'sympathy_and_support',
                       'sympathy_and_support')

alt.LayerChart(...)

### Get the most frequent location pair

In [38]:
loc_pairs = top_location_pairs(df_loc_graph, 'sympathy_and_support', top_pairs_count=1)
loc_pairs

[['California’s', 'California']]

### Show a sample for this location pair

In [39]:
show_tweets_locations(df_tweets, class_label='sympathy_and_support',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"Heartbreaking images of the #McKinneyFire, now California’s largest #wildfire of 2022, and the #OakFire near #Yosemite. They say the Oak Fire fire is near 70% containment…God speed to all the firefighters in Northern #California today. Sending love to all. https://t.co/2zzQVO...","[California’s, Oak Fire, Northern, California]"
1,"Heartbreaking images of the #McKinneyFire, now California’s largest #wildfire of 2022, and the #OakFire near #Yosemite. They say the Oak Fire fire is near 70% containment…God speed to all the firefighters in Northern #California today. Sending love to all. https://t.co/2zzQVO...","[California’s, Oak Fire, Northern, California]"
2,"Heartbreaking images of the #McKinneyFire, now California’s largest #wildfire of 2022, and the #OakFire near #Yosemite. They say the Oak Fire fire is near 70% containment…God speed to all the firefighters in Northern #California today. Sending love to all. https://t.co/2zzQVO...","[California’s, Oak Fire, Northern, California]"
3,"Heartbreaking images of the #McKinneyFire, now California’s largest #wildfire of 2022, and the #OakFire near #Yosemite. They say the Oak Fire fire is near 70% containment…God speed to all the firefighters in Northern #California today. Sending love to all. https://t.co/2zzQVO...","[California’s, Oak Fire, Northern, California]"
4,"Heartbreaking images of the #McKinneyFire, now California’s largest #wildfire of 2022, and the #OakFire near #Yosemite. They say the Oak Fire fire is near 70% containment…God speed to all the firefighters in Northern #California today. Sending love to all. https://t.co/2zzQVO...","[California’s, Oak Fire, Northern, California]"


In [40]:
plot_disaster_mentions(df_loc_graph, 'infrastructure_and_utility_damage',
                       'infrastructure_and_utility_damage')

alt.LayerChart(...)

### Get the most frequent location pair

In [41]:
loc_pairs = top_location_pairs(df_loc_graph, 'infrastructure_and_utility_damage', top_pairs_count=1)
loc_pairs

[['Northern California', 'California']]

### Show a sample for this location pair

In [42]:
show_tweets_locations(df_tweets, class_label='infrastructure_and_utility_damage',
                      locations=loc_pairs[0], sample=5)

,tweet_text,location
0,"California's largest wildfire of the year has left a long trail of destruction.\n\nThe McKinney Fire in Northern California erupted on Friday — it's now nearly 87 square miles in size, according to US firefighting officials. https://t.co/euF0NgeA1p","[California, Northern California, US]"
1,RT @dwnews: California's largest wildfire of the year has left a long trail of destruction.\n\nThe McKinney Fire in Northern California erupt…,"[California, Northern California]"
2,RT @dwnews: California's largest wildfire of the year has left a long trail of destruction.\n\nThe McKinney Fire in Northern California erupt…,"[California, Northern California]"
3,RT @dwnews: California's largest wildfire of the year has left a long trail of destruction.\n\nThe McKinney Fire in Northern California erupt…,"[California, Northern California]"
4,"California's largest wildfire of the year has left a long trail of destruction.\n\nThe McKinney Fire in Northern California erupted on Friday — it's now nearly 87 square miles in size, according to US firefighting officials. https://t.co/euF0NgeA1p","[California, Northern California, US]"
